### Parametric synthesis (pareto front)

In [ ]:
# === Block 3: Multi‐objective optimization with pymoo ===

import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
# replace factory imports with direct operator imports
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.visualization.scatter import Scatter
from pymoo.core.repair import Repair


# scenarios: three loading cases
tasks = [
    {"displacement": 5e-3,  "z": 15e-3},
    {"displacement": 15e-3, "z": 35e-3},
    {"displacement": 10e-3, "z": 55e-3}
]

# number of ribs in design
n_rib = 3

# total number of decision variables: [th_front, th_back, xf, zf, phi_1..phi_n, th_rib1..th_ribn]
total_vars = 4 + 2 * n_rib

n_springs_front = 20
n_springs_back  = 20
n_springs_rib   = 12
L0 = 70e-3
X1 = 35e-3
beam_width = 35e-3
moving_cylinder_radius = 15e-3

def angle_constraints(X, n_rib):
    # X = [th_front, th_back, xf, zf, phi1..phi_n, th_rib1..th_rib_n]
    xf, zf = X[2], X[3]
    phis = X[4:4+n_rib]
    # Здесь ваша логика расчёта φ_b, φ_a из xf,zf:
    phi_a = np.rad2deg(np.arctan((L0 - zf) / xf))
    phi_b = np.rad2deg(-np.arctan(zf / (X1 + xf)))
    
    # Для каждого ребра два неравенства:
    lower = phis - phi_b             # phi_i ≥ phi_b
    upper = phi_a - phis             # phi_i ≤ phi_a
    
    print(f"φ_b = {phi_b:.3f}°, φ_a = {phi_a:.3f}°")
    for i, phi in enumerate(phis):
        print(f" R{i}: φ={phi:.3f}° → lower={phi-phi_b:.3f}, upper={phi_a-phi:.3f}")
    
    return np.hstack([lower, upper])

def phi_constr_cone(xf, t, phi):
    if xf <= 0:
        return 0.0, 0.0
    # Расстояние от фокуса до ребра «вдоль» оси
    a = xf / np.cos(phi)
    # Горизонтальная «полуширина» ребра
    b = (t / 2) / np.cos(phi)
    # Длины вспомогательных сторон треугольников
    c = np.sqrt(a**2 + b**2 - 2*a*b*np.cos(np.pi/2 - phi))
    d = np.sqrt(a**2 + b**2 - 2*a*b*np.cos(np.pi/2 + phi))
    # Угловые допуски по формулам косинусов
    delta_minus = np.arccos((a**2 + c**2 - b**2) / (2*a*c))
    delta_plus  = np.arccos((a**2 + d**2 - b**2) / (2*a*d))
    return delta_minus, delta_plus

def nonintersect_constraints(X, n_rib):
    if n_rib < 2:
        return np.array([], dtype=float)

    xf, zf = X[2], X[3]
    phis    = np.deg2rad(X[4:4+n_rib])
    ths     = X[4+n_rib:4+2*n_rib]
    # 1) сортируем
    order = np.argsort(phis)
    phs   = phis[order]
    ths   = ths[order]

    # 2) расчёт δ
    delta_minus = np.zeros(n_rib)
    delta_plus  = np.zeros(n_rib)
    for i in range(n_rib):
        dm, dp = phi_constr_cone(xf, ths[i], phs[i])
        delta_minus[i], delta_plus[i] = dm, dp

    # 3) gᵢ ≥ 0: если все хорошо, возвращаем строго положительные eps
    g = np.empty(n_rib-1)
    for i in range(n_rib-1):
        g[i] = (phs[i+1] - delta_minus[i+1]) - (phs[i] + delta_plus[i])
        # небольшая подтяжка в плюсовую зону, чтобы не было нулей
        g[i] = g[i] if g[i] >= 0 else g[i]
    return g

def build_optimization(n_rib):
    # Initial guesses
    th_front0 = 2e-3
    th_back0  = 2e-3
    xf0       = 10e-3
    zf0       = 10e-3
    
    phi_a = np.rad2deg(np.arctan((L0 - zf0) / xf0))
    phi_b = np.rad2deg(-np.arctan(zf0 / (X1 + xf0)))

    if n_rib > 0:
        phi0 = np.linspace(phi_b, phi_a, n_rib)
    else:
        phi0 = np.array([])
    
    th_ribs0  = np.ones(n_rib) * 2e-3
    X_init    = np.hstack([th_front0, th_back0, xf0, zf0, phi0, th_ribs0])

    # Bounds
    bnds = []
    bnds += [(1e-3, 5e-3), (1e-3, 5e-3)]      # thickness_front, thickness_back
    bnds += [(1e-3, 1000e-3), (-1000e-3, 1000e-3)]         # focal coords xf, zf
    bnds += [(-45,45)] * n_rib  # rib angles phi
    bnds += [(1e-3, 2e-3)] * n_rib               # rib thickness

    return X_init, bnds

# build bounds and initial guess (from Block 2 helper)
X0, bnds = build_optimization(n_rib)

class FinrayMOO(ElementwiseProblem):
    def __init__(self):
        xl = np.array([b[0] for b in bnds])
        xu = np.array([b[1] for b in bnds])
        super().__init__(n_var=total_vars,
                         n_obj=3,
                         n_constr=2 * n_rib + (n_rib - 1),
                         xl=xl, xu=xu)

    def _evaluate(self, X, out, *args, **kwargs):
        # unpack design
        th_front, th_back, xf, zf = X[:4]
        phi = X[4:4+n_rib]
        th_ribs = X[4+n_rib:].tolist()  # convert to list.tolist()  # convert to Python list to avoid numpy truth ambiguity

        # accumulate metrics across scenarios
        Fx_tot = Fz_tot = S_tot = Ef_tot = Eb_tot = 0.0

            
        _, _, Fx, Fz, _, _, S, Ef, Eb = run_finray_simulation(
            rib_angles_deg=phi,
            thickness_ribs=th_ribs,
            n_springs_front=n_springs_front,
            n_springs_back=n_springs_back,
            n_springs_rib=n_springs_rib,
            thickness_front=th_front,
            thickness_back=th_back,
            beam_width=beam_width,
            L0=L0,
            X1=X1,
            xf=xf,
            zf=zf,
            moving_cylinder_x=moving_cylinder_radius + th_front/2,
            moving_cylinder_z=15e-3,
            moving_cylinder_radius=moving_cylinder_radius,
            moving_cylinder_displacement=5e-3,
            vis=False
        )
        
        _, _, Fx, Fz, _, _, S, Ef, Eb = run_finray_simulation(
            rib_angles_deg=phi,
            thickness_ribs=th_ribs,
            n_springs_front=n_springs_front,
            n_springs_back=n_springs_back,
            n_springs_rib=n_springs_rib,
            thickness_front=th_front,
            thickness_back=th_back,
            beam_width=beam_width,
            L0=L0,
            X1=X1,
            xf=xf,
            zf=zf,
            moving_cylinder_x=moving_cylinder_radius + th_front/2,
            moving_cylinder_z=35e-3,
            moving_cylinder_radius=moving_cylinder_radius,
            moving_cylinder_displacement=15e-3,
            vis=False
        )
        
        _, _, Fx, Fz, _, _, S, Ef, Eb = run_finray_simulation(
            rib_angles_deg=phi,
            thickness_ribs=th_ribs,
            n_springs_front=n_springs_front,
            n_springs_back=n_springs_back,
            n_springs_rib=n_springs_rib,
            thickness_front=th_front,
            thickness_back=th_back,
            beam_width=beam_width,
            L0=L0,
            X1=X1,
            xf=xf,
            zf=zf,
            moving_cylinder_x=moving_cylinder_radius + th_front/2,
            moving_cylinder_z=55e-3,
            moving_cylinder_radius=moving_cylinder_radius,
            moving_cylinder_displacement=10e-3,
            vis=False
        )
        
        Fx_tot += np.sum(Fx)
        Fz_tot += np.sum(Fz)
        S_tot  += S
        Ef_tot += Ef
        Eb_tot += Eb

        # objectives to minimize:
        # f1: negative lift‐to‐drag ratio => maximize Fz/Fx
        f1 = - Fz_tot / (Fx_tot + 1e-9)
        # f2: average contact area
        f2 = S_tot / len(tasks)
        # f3: inverse energy efficiency => minimize Ef/Eb
        f3 = (Ef_tot + 1e-9) / (Eb_tot + 1e-9)

        # constraints g(x) <= 0: combine angle and nonintersection (which must be >=0)
        g_angle = angle_constraints(X, n_rib)
        g_nonint = nonintersect_constraints(X, n_rib)
        
        print("===== Constraint check =====")
        for i, val in enumerate(g_angle):
            status = "OK" if val >= 0 else "VIOLATED"
            print(f"  angle constr[{i:2d}] = {val:.4e} ({status})")
        for i, val in enumerate(g_nonint):
            status = "OK" if val >= 0 else "VIOLATED"
            print(f" nonint constr[{i:2d}] = {val:.4e} ({status})")
        
        out['F'] = [f1, f2, f3]
        out['G'] = np.hstack([-g_angle, -g_nonint])

class FinrayRepair(Repair):
    def _do(self, problem, pop, **kwargs):
        # Извлекаем все решения из популяции
        X = pop.get("X")
        for k, x in enumerate(X):
            xf, zf = float(x[2]), float(x[3])
            # Вычисляем текущие φ_b и φ_a
            phi_a = np.rad2deg(np.arctan((L0 - zf) / xf))
            phi_b = np.rad2deg(-np.arctan(zf / (X1 + xf)))
            # Вычисляем индексы угловой части и «ремонтируем» её
            start, end = 4, 4 + problem.n_rib
            phis = np.clip(x[start:end], phi_b, phi_a)
            # Записываем обратно в хромосому
            x[start:end] = phis
            pop[k].X = x
        return pop

algorithm = NSGA2(
    pop_size=200,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, prob_var=0.9, eta=15),
    mutation=PolynomialMutation(eta=20),
    eliminate_duplicates=True,
    repair=FinrayRepair()    # добавляем наш Repair-оператор
)

res = minimize(
    problem=FinrayMOO(), 
    algorithm=algorithm,
    termination=('n_gen', 50),
    seed=43,
    verbose=True
)

# extract Pareto solutions
pareto_F = res.F
pareto_X = res.X
#asdasdasdtffrgf

### Finray parametric synthesis (one reward)

In [ ]:
import mujoco
import mujoco_viewer
import numpy as np
import time
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint
import tempfile

n_springs_front = 20
n_springs_back  = 20
n_springs_rib   = 12
L0 = 70e-3
X1 = 35e-3
beam_width = 35e-3

moving_cylinder_radius = 20e-3
moving_cylinder_displacement = 15e-3

moving_cylinder_z = 35e-3

def aggregate_metrics(winding_number, area_factor, sum_fx, sum_fz,
                      bounds):

    fx_min, fx_max = bounds['fx']
    fz_min, fz_max = bounds['fz']
    
    # Maps to [0; 1]
    def norm_max(x, x_min, x_max):
        return (x - x_min) / (x_max - x_min)
       
    nfx = norm_max(sum_fx, fx_min, fx_max)
    nfz = norm_max(sum_fz, fz_min, fz_max)

    print(f"mormalized sum FX: {nfx}")
    print(f"mormalized sum FZ: {nfz}")
    
    total_score = - winding_number - area_factor - nfx + nfz
    return total_score

def get_best_finray(X):
    th_front, th_back, xf, zf = X[:4]
    n_rib = (len(X) - 4) // 2
    phi_ribs = X[4 : 4 + n_rib].tolist()
    th_ribs  = X[4 + n_rib :].tolist()
    
    # Run simulation
    pos_init, pos_deform, con_Fx, con_Fz, con_Px, con_Pz, con_S, E_front, E_back, valid_rib_indices = run_finray_simulation(
        rib_angles_deg = phi_ribs,
        thickness_ribs = th_ribs,
        n_springs_front = n_springs_front,
        n_springs_back  = n_springs_back,
        n_springs_rib   = n_springs_rib,
        thickness_front = th_front,
        thickness_back  = th_back,
        beam_width      = beam_width,
        L0              = L0,
        X1              = X1,
        xf              = xf,
        zf              = zf,
        moving_cylinder_x           = moving_cylinder_radius + th_front / 2,
        moving_cylinder_z           = moving_cylinder_z,
        moving_cylinder_radius      = moving_cylinder_radius,
        moving_cylinder_displacement = moving_cylinder_displacement,
        vis             = False
    )
    
    fx_arr = np.sum(np.array(con_Fx))
    fz_arr = np.sum(np.array(con_Fz))
    S_arr = np.array(con_S)
    S_max = L0 * beam_width
    area_factor = S_arr/S_max
    
    metric_bounds = {
    'fx':      (0, 50.0),
    'fz':      (0, 10.0),
    }

    angles = []
    # Winding number
    angles = np.arctan2(pos_deform[:,1], pos_deform[:,0])
    angles_unwrapped = np.unwrap(angles)
    delta_angle = angles_unwrapped[-1] - angles_unwrapped[0]
    winding_number = delta_angle / (2 * np.pi)

    print("---------------------------------------")
    print(f"Winding number: {winding_number}")
    print(f"Area factor: {S_arr/S_max}")
    print(f"Sum(FX): {fx_arr}")
    print(f"Sum(FZ): {fz_arr}")
    
    reward = aggregate_metrics(
        winding_number, area_factor, fx_arr, np.abs(fz_arr),
        bounds=metric_bounds
    )
    
    print(f"reward = {reward}")
    
    return reward

def build_optimization(n_rib):
    # Initial guesses
    th_front0 = 2e-3
    th_back0  = 2e-3
    xf0       = 10e-3
    zf0       = 35e-3
    
    phi_a = np.rad2deg(np.arctan((L0 - zf0) / xf0))
    phi_b = np.rad2deg(-np.arctan(zf0 / (X1 + xf0)))

    if n_rib > 0:
        phi0 = np.linspace(phi_b, phi_a, n_rib)
    else:
        phi0 = np.array([])
    
    th_ribs0  = np.ones(n_rib) * 2e-3
    X_init    = np.hstack([th_front0, th_back0, xf0, zf0, phi0, th_ribs0])

    # Bounds
    bnds = []
    bnds += [(2e-3, 3e-3), (3e-3, 5e-3)]      # thickness_front, thickness_back
    bnds += [(1e-3, 2000e-3), (-1000e-3, 1000e-3)]         # focal coords xf, zf
    bnds += [(-60, 60)] * n_rib  # rib angles phi
    bnds += [(2e-3, 3e-3)] * n_rib               # rib thickness

    return X_init, bnds

def angle_constraints(X, n_rib):
    # X = [th_front, th_back, xf, zf, phi1..phi_n, th_rib1..th_rib_n]
    xf, zf = X[2], X[3]
    phis = X[4:4+n_rib]
    # Здесь ваша логика расчёта φ_b, φ_a из xf,zf:
    phi_a = np.rad2deg(np.arctan((L0 - zf) / xf))
    phi_b = np.rad2deg(-np.arctan(zf / (X1 + xf)))
    
    # Для каждого ребра два неравенства:
    lower = phis - phi_b             # phi_i ≥ phi_b
    upper = phi_a - phis             # phi_i ≤ phi_a
    return np.hstack([lower, upper])

def phi_constr_cone(xf, t, phi):
    if xf <= 0:
        return 0.0, 0.0
    # Расстояние от фокуса до ребра «вдоль» оси
    a = xf / np.cos(phi)
    # Горизонтальная «полуширина» ребра
    b = (t / 2) / np.cos(phi)
    # Длины вспомогательных сторон треугольников
    c = np.sqrt(a**2 + b**2 - 2*a*b*np.cos(np.pi/2 - phi))
    d = np.sqrt(a**2 + b**2 - 2*a*b*np.cos(np.pi/2 + phi))
    # Угловые допуски по формулам косинусов
    delta_minus = np.arccos((a**2 + c**2 - b**2) / (2*a*c))
    delta_plus  = np.arccos((a**2 + d**2 - b**2) / (2*a*d))
    return delta_minus, delta_plus

def nonintersect_constraints(X, n_rib):
    if n_rib < 2:
        return np.array([], dtype=float)

    xf, zf = X[2], X[3]
    phis    = np.deg2rad(X[4:4+n_rib])
    ths     = X[4+n_rib:4+2*n_rib]
    # 1) сортируем
    order = np.argsort(phis)
    phs   = phis[order]
    ths   = ths[order]

    # 2) расчёт δ
    delta_minus = np.zeros(n_rib)
    delta_plus  = np.zeros(n_rib)
    for i in range(n_rib):
        dm, dp = phi_constr_cone(xf, ths[i], phs[i])
        delta_minus[i], delta_plus[i] = dm, dp

    # 3) gᵢ ≥ 0: если все хорошо, возвращаем строго положительные eps
    g = np.empty(n_rib-1)
    for i in range(n_rib-1):
        g[i] = (phs[i+1] - delta_minus[i+1]) - (phs[i] + delta_plus[i])
        # небольшая подтяжка в плюсовую зону, чтобы не было нулей
        g[i] = g[i] if g[i] >= 0 else g[i]
    return g

history = []

def de_callback(xk, convergence):
    fval = get_best_finray(xk)
    history.append((xk.copy(), fval))
    # возвращаем False, чтобы DE не прерывался досрочно
    return False

n_rib = 3

X_init, bnds = build_optimization(n_rib)
print(f"Search space of {len(X_init)} parameters")

nlc_angle = NonlinearConstraint(
    fun=lambda X: angle_constraints(X, n_rib),
    lb=0, ub=np.inf
)

nlc_nonint = NonlinearConstraint(
    fun=lambda X: nonintersect_constraints(X, n_rib),
    lb=0.0,  # g_i >= 0
    ub=np.inf,
    keep_feasible=True
)

constraints = (nlc_angle, nlc_nonint)

from multiprocessing import Pool

res = differential_evolution(
    get_best_finray,
    bounds = bnds,
    constraints = constraints,
    maxiter = 100,
    popsize = len(X_init) * 100,
    mutation = (0.5,1),
    recombination = 0.7,
    updating = 'immediate',
    disp = True,
    polish = False
)

print('Optimal result:', res)
optimal_X = res.x

opt = res.x
th_front_opt   = opt[0]
th_back_opt    = opt[1]
xf_opt         = opt[2]
zf_opt         = opt[3]
phi_ribs_opt   = opt[4 : 4 + n_rib]
th_ribs_opt    = opt[4 + n_rib : 4 + 2*n_rib]



print(f"t_front = {th_front_opt*1e3}")
print(f"t_back = {th_back_opt*1e3}")
print(f"t_ribs = {th_ribs_opt*1e3}")
print(f"phi_ribs = {phi_ribs_opt}")


print(f"xf = {xf_opt*1e3}")
print(f"zf = {zf_opt*1e3}")

th_ribs_opt = list(th_ribs_opt) if th_ribs_opt is not None else []

start_time = time.time()

pos_init, pos_deform, con_Fx, con_Fz, con_Px, con_Pz, con_S, E_front, E_back, valid_rib_indices = run_finray_simulation(
    rib_angles_deg = phi_ribs_opt,
    thickness_ribs = th_ribs_opt,
    n_springs_front = n_springs_front,
    n_springs_back = n_springs_back,
    n_springs_rib = n_springs_rib,
    thickness_front = th_front_opt,
    thickness_back = th_back_opt,
    beam_width = beam_width,
    L0 = L0,
    X1 = X1,
    xf = xf_opt,
    zf = zf_opt,
    moving_cylinder_x = moving_cylinder_radius + th_front_opt/2,
    moving_cylinder_z = moving_cylinder_z + 5e-3,
    moving_cylinder_radius = moving_cylinder_radius,
    moving_cylinder_displacement = moving_cylinder_displacement,
    vis = True
)


fx_arr = np.sum(np.array(con_Fx))
fz_arr = np.sum(np.array(con_Fz))
S_arr = np.array(con_S)
S_max = L0 * beam_width
area_factor = S_arr/S_max

metric_bounds = {
'fx':      (0, 50.0),
'fz':      (0, 10.0),
}

angles = []
# Winding number
angles = np.arctan2(pos_deform[:,1], pos_deform[:,0])
angles_unwrapped = np.unwrap(angles)
delta_angle = angles_unwrapped[-1] - angles_unwrapped[0]
winding_number = delta_angle / (2 * np.pi)

print("---------------------------------------")
print(f"Winding number: {winding_number}")
print(f"Area factor: {S_arr/S_max}")
print(f"Sum(FX): {fx_arr}")
print(f"Sum(FZ): {fz_arr}")

reward = aggregate_metrics(
    winding_number, area_factor, fx_arr, np.abs(fz_arr),
    bounds=metric_bounds
)

print(f"reward = {reward}") 
